In [1]:
import numpy as np
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from PIL import Image

In [2]:
template_file = ""
mel_string = "MEL"
hg_string = "HG"
epth_string = "EPTH"

In [3]:
with open("template.pbrt", "r") as f:
    template_file = f.read()

assert template_file != ""

In [4]:
pbrt_renders = Path('pbrt-outputs')

In [5]:
bases = np.zeros((1000,80,80,3), dtype="float32")
points = np.zeros((1000,3))

In [6]:
for i, item in enumerate(pbrt_renders.iterdir()):
    if item.is_file():
        f = item.name
        m = f.index('m')
        l = f.index('l')
        h = f.index('h')
        g = f.index('g')
        p = f.index('p')

        points[i,0] = float(f[:m])
        points[i,1] = float(f[l+2:h])
        points[i,2] = float(f[g+2:p-1])

        bases[i,:] = cv2.imread(f'{item}', cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)


In [7]:
for subj_id in range(23):
    output_data = Path(f'output-data/S{subj_id:03d}')
    mel_paths = sorted(output_data.glob('**/out-mel/*.txt'))
    hg_paths = sorted(output_data.glob('**/out-hem/*.txt'))
    epth_paths = sorted(output_data.glob('**/out-epth/*.txt'))
    
    mels = [np.loadtxt(f, delimiter=',') for f in mel_paths]
    hgs = [np.loadtxt(f, delimiter=',') for f in hg_paths]
    epths = [np.loadtxt(f, delimiter=',') for f in epth_paths]

    n = len(mels)
    
    Path(f"interpolated-patches/S{subj_id:03d}").mkdir(parents=True, exist_ok=True)

    for i in range(n):
        patch_mels = mels[i]
        patch_hgs = hgs[i]
        patch_epths = epths[i]

        h, w = patch_mels.shape

        interpolated = np.zeros((h,w,3), dtype='float32')

        for row in range(h):
            for col in range(w):
                mel_p = patch_mels[row,col]
                hg_p = patch_hgs[row,col]
                epth_p = patch_epths[row,col] * 0.001

                sorted_points = points[np.argsort(cdist(points,[[mel_p,hg_p,epth_p]]).squeeze())]
                f, g, b, c = sorted_points[:4]

                d_pf = cdist([f], [[mel_p, hg_p, epth_p]])
                d_pg = cdist([g], [[mel_p, hg_p, epth_p]])
                d_pb = cdist([b], [[mel_p, hg_p, epth_p]])
                d_pc = cdist([c], [[mel_p, hg_p, epth_p]])
                d_t = d_pf + d_pg + d_pb + d_pc

                w_f = (d_pf/d_t)
                w_g = (d_pg/d_t)
                w_b = (d_pb/d_t)
                w_c = (d_pc/d_t)

                basis_f = cv2.resize(bases[np.argwhere(points == f)[0][0]], (w,h), interpolation=cv2.INTER_AREA)
                basis_g = cv2.resize(bases[np.argwhere(points == g)[0][0]], (w,h), interpolation=cv2.INTER_AREA)
                basis_b = cv2.resize(bases[np.argwhere(points == b)[0][0]], (w,h), interpolation=cv2.INTER_AREA)
                basis_c = cv2.resize(bases[np.argwhere(points == c)[0][0]], (w,h), interpolation=cv2.INTER_AREA)

                
                interpolated[row,col,:] = (w_f * basis_f[row,col,:]) + (w_g * basis_g[row,col,:]) + (w_b * basis_b[row,col,:]) + (w_c * basis_c[row,col,:])
                
        cv2.imwrite(f"interpolated-patches/S{subj_id:03d}/{i:03d}.exr", interpolated)